In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='Blues')
from slmsuite.holography.algorithms import Hologram, SpotHologram
from tools.IonChainTools import *

In [ ]:
def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200):
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

def phase_calculate(target_size,pixel_size,ions,N,v,cameraslm, plot = True):
    '''
    target size: the size of target pattern, here is the number of pixels of SLM;
    pixel_size: the pixel size of camera;
    ions: target ions;
    N: number of ions;
    v: trap frequency;
    return: the farfield amplitude
    '''
    mag = 100 * 7.8 / 2.2
    position = np.array(ion_position_cam(N,v,pixel_size,magnititude= mag))
    dy = target_size[0]//8 
    pos = position[ions]
    position_cam = np.column_stack((pos+target_size[1]//2,np.full(pos.shape,target_size[0]//2+dy)))
    print(position_cam)
    position_cam = position_cam.T
    all = range(N)
    no_ions = np.array([ i for i in all if i not in ions])
    null = position[no_ions]
    null_points_cam = np.column_stack((null+target_size[1]//2,np.full(null.shape, target_size[0]//2+dy)))
    print(null_points_cam)
    null_points_cam = null_points_cam.T
    holo = SpotHologram(shape=(1200,1920), 
            spot_vectors=position_cam, 
            null_vectors=null_points_cam, 
            null_radius=60,  
            cameraslm=cameraslm)
    holo.optimize(method = 'WGS-Kim', maxiter=50)
    if plot == True:
        holo.plot_nearfield(padded=True,cbar=True)
        limit = np.array(((500,1400),(600,900)))
        holo.plot_farfield(cbar=True,limits = limit,figsize=(10, 8))
    
    image = holo.extract_farfield()
        
    return image

In [ ]:
target_size = (1200,1920)
pixel_size = np.array([7.8e-6,7.8e-6]) 
ions = np.array([0,1,2,5])

In [ ]:
N = 6
v = 2*π*1e6 
slm_size = target_size
#Create a virtual SLM and camera
from slmsuite.hardware.slms.slm import SLM
from slmsuite.hardware.cameras.camera import Camera
from slmsuite.hardware.cameraslms import FourierSLM

wav_um = 0.730
slm = SLM (slm_size[1],slm_size[0], dx_um=7.8, dy_um=7.8, wav_um=wav_um)
camera = Camera(target_size[1],target_size[0])

slm.set_measured_amplitude_analytic(800)
fs = FourierSLM(camera, slm)

In [ ]:
image = phase_calculate(target_size,pixel_size,ions,N,v,fs,plot = True)

In [ ]:
#plot the enhanced farfield amplitude
amp = np.abs(image)
# norm_amp = (amp - amp.min())/(amp.max() - amp.min())
log_scale_data = np.log(1+np.sqrt(amp))
from matplotlib import ticker
plt.figure(figsize=(8, 6))
img = plt.imshow(log_scale_data)
cb = plt.colorbar(img, label='Amplitude', format='%.2f')
# Adjust the colorbar labels to show 10^log_data format
cb.formatter = ticker.FuncFormatter(lambda x, pos: f'10^{x:.2f}')
cb.update_ticks()
plt.title('Farfield Data Visualization')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

In [ ]:
#calculate the intensity
intensity = amp**2
cal = []
mag = 100 * 7.8 / 2.2
position = np.array(ion_position_cam(N,v,pixel_size,magnititude= mag))
dy = target_size[0]//8 
all = range(N)
pos_all = position[all]
position_all = np.column_stack((np.full(pos_all.shape,target_size[0]//2+dy),pos_all+target_size[1]//2))
for i, pos in enumerate(position_all):
    x = int(pos[0])
    y = int(pos[1])
    cal.append(np.sum(intensity[x-40:x+40, y-40:y+40]))

In [ ]:
#plot the intensity
plt.scatter(range(6),cal, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

In [ ]:
#calculate the fidelity
intensity = cal
background = 0
inten = np.array([ i- background for i in intensity])
all = range(N)
undesire = [ i for i in all if i not in ions]
Omega_desire = np.sqrt(inten[ions])
Omega_undesire = np.sqrt(inten[undesire])
F1 = sum(Omega_desire)/(len(ions)*max(Omega_desire))
F2 = 1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire))
F = F1*F2

print(F1)
print(F2)
print(F)